bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount fips

In [1]:
from env import get_db_url
import pandas as pd
import numpy as np
import os

In [2]:
#performs SQL query if file does not already exist
if ~os.path.isfile('data/zillow.csv'):
    url = get_db_url('zillow')
    query = '''
    SELECT bedroomcnt,
    bathroomcnt,calculatedfinishedsquarefeet,
    taxvaluedollarcnt,yearbuilt,
    taxamount, fips 
    FROM properties_2016 
    JOIN propertylandusetype
    USING(propertylandusetypeid)
    WHERE propertylandusedesc = "Single Family Residential";'''
    #query for properties_2016 table
    df_2016 = pd.read_sql(query,url)
    query = '''
    SELECT bedroomcnt,
    bathroomcnt,calculatedfinishedsquarefeet,
    taxvaluedollarcnt,yearbuilt,
    taxamount, fips 
    FROM properties_2017
    JOIN propertylandusetype
    USING(propertylandusetypeid)
    WHERE propertylandusedesc = "Single Family Residential";'''
    #query for properties_2017 and concat with properties_2016 into final dataframe
    df = pd.concat([df_2016,pd.read_sql(query,url)],axis=0)
    df.to_csv('data/zillow.csv')
else:
    df = pd.read_csv('data/zillow.csv')

KeyboardInterrupt: 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4301225 entries, 0 to 4301224
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 229.7 MB


## Rename Columns in snake case

In [ ]:
df = df.rename(columns={'bedroomcnt':'bed_count','bathroomcnt':'bath_count',\
    'taxvaluedollarcnt':'tax_value','yearbuilt':'year_built','taxamount':'taxes',\
        'calculatedfinishedsquarefeet':'calc_finished_sqft'})

In [ ]:
print(f'df shape: {df.shape[0]} rows x {df.shape[1]} columns')
df.isna().sum()

df shape: 4301225 rows x 7 columns


bedroomcnt                         22
bathroomcnt                        22
calculatedfinishedsquarefeet    18315
taxvaluedollarcnt                 981
yearbuilt                       19990
taxamount                        8838
fips                                0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4301225 entries, 0 to 4301224
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 229.7 MB


dropping indices where $null \gt 4$ (i.e. where more than half the features are null)

In [ ]:
df = df.dropna(subset=['bed_count','bath_count']).dropna(thresh=4)
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,4.0,3.0,2090.0,615000.0,1973.0,8625.75,6037.0
1,5.0,3.0,2479.0,531642.0,1973.0,6722.67,6037.0
2,4.0,2.0,2057.0,328148.0,1973.0,4160.63,6037.0
3,4.0,3.0,2423.0,297377.0,1978.0,3883.47,6037.0
4,4.0,3.0,2226.0,297001.0,1978.0,3884.27,6037.0


In [ ]:
#check if there are half bedrooms (should not be any)
df[(df.bedroomcnt / .5)%2 > 0]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips


There are none, so we will convert the bedroomcnt to int

In [ ]:
df.bedroomcnt = df.bedroomcnt.astype(int)
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4300579 entries, 0 to 4301224
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    4300579 non-null  int64  
 1   bathroomcnt                   4300579 non-null  float64
 2   calculatedfinishedsquarefeet  4282902 non-null  float64
 3   taxvaluedollarcnt             4300222 non-null  float64
 4   yearbuilt                     4281221 non-null  float64
 5   taxamount                     4292383 non-null  float64
 6   fips                          4300579 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 262.5 MB


In [ ]:
#We'll use 0 as the default value in yearbuilt and convert to int
df.yearbuilt = df.yearbuilt.fillna(0).astype(int)
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4300579 entries, 0 to 4301224
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    4300579 non-null  int64  
 1   bathroomcnt                   4300579 non-null  float64
 2   calculatedfinishedsquarefeet  4282902 non-null  float64
 3   taxvaluedollarcnt             4300222 non-null  float64
 4   yearbuilt                     4300579 non-null  int64  
 5   taxamount                     4292383 non-null  float64
 6   fips                          4300579 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 262.5 MB


Because `taxvaluedollarcnt` has less than 1% na, we will drop this as well

In [ ]:
df = df.dropna(subset=['taxvaluedollarcnt','taxamount'])
df.isna().sum()


bedroomcnt                          0
bathroomcnt                         0
calculatedfinishedsquarefeet    14776
taxvaluedollarcnt                   0
yearbuilt                           0
taxamount                           0
fips                                0
dtype: int64

checking what percentage of the data are NaN values in calculatedfinishedsquarefeet

In [ ]:
df = df.dropna(subset='calculatedfinishedsquarefeet')
df.shape[0]/datashape[0]

0.9944713424663904

In [ ]:
df[df.calc_finished_sqft.astype(int) < df.calc_finished_sqft]

,bed_count,bath_count,calc_finished_sqft,tax_value,year_built,taxes,fips


In [ ]:
#check that the wrangle function works
import wrangle
df = wrangle.wrangle_zillow()
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4277445 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   bed_count           4277445 non-null  int64  
 1   bath_count          4277445 non-null  float64
 2   calc_finished_sqft  4277445 non-null  float64
 3   tax_value           4277445 non-null  float64
 4   year_built          4277445 non-null  int64  
 5   taxes               4277445 non-null  float64
 6   fips                4277445 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 261.1 MB
